In [1]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
import nltk
import wikipedia
from lxml import html
headers = {
    "User-Agent": "Mozilla/5.0"
}
BASE_URL = "https://en.wikipedia.org/wiki/"

In [2]:
def search_wikipedia(movie_name):
    search_url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "list": "search",
        "srsearch": movie_name + " film",
        "format": "json"
    }

    r = requests.get(search_url, params=params, headers=headers)
    data = r.json()

    if data["query"]["search"]:
        return data["query"]["search"][0]["title"]  # best match
    else:
        return None

# --------------------------
# Get plot from page
# --------------------------
def get_plot_from_title(title):
    url = f"https://en.wikipedia.org/wiki/{title.replace(' ', '_')}"
    page = wikipedia.WikipediaPage(title=title)
    text = page.section(section_title="Plot")
    return text









In [3]:

df = pd.read_csv("malayalam_movie_cast_dataset.csv")
df  = df[["movie_name"]].drop_duplicates()
plots = []

for i, row in df.iterrows():
    try:
        movie = row["movie_name"]
        new_name = search_wikipedia(movie)
        print(f"Scraping plot for: {movie}")
        plot = get_plot_from_title(new_name)
        print("new name is",new_name)
        #print("plot is",plot)
    except:
        plot=""
    plots.append(plot)
    time.sleep(1)  # polite delay
        

df["plot"] = plots
df.to_csv("movies_with_plot.csv", index=False)

Scraping plot for: Sufi Paranja Katha
new name is Sufi Paranja Katha (film)
Scraping plot for: Thanthonni
new name is Thanthonni
Scraping plot for: Kadaksham
new name is Kadaksham
Scraping plot for: T. D. Dasan Std. VI B
new name is T. D. Dasan Std. VI B
Scraping plot for: Alexander The Great
new name is Alexander the Great (1956 film)
Scraping plot for: Kadha Thudarunnu
new name is Kadha Thudarunnu
Scraping plot for: Oru Naal Varum
new name is Oru Naal Varum
Scraping plot for: Malarvaadi Arts Club
new name is Malarvaadi Arts Club
Scraping plot for: Kutty Srank
new name is Kutty Srank
Scraping plot for: Avan
new name is Avan Jogia
Scraping plot for: Taskkara Lahala
new name is List of Malayalam films of 2010
Scraping plot for: Raama Raavanan
new name is Raama Raavanan
Scraping plot for: Plus Two
new name is Plus Two (film)
Scraping plot for: Paattinte Palazhy
new name is Paattinte Palazhy
Scraping plot for: Aathmakatha
new name is Aathmakatha
Scraping plot for: 3 Char Sau Bees
new name

c:\Users\jerar\anaconda3\envs\castnet\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\jerar\anaconda3\envs\castnet\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Scraping plot for: Theerppu
new name is Theerppu
Scraping plot for: Peace
new name is War and Peace (film series)
Scraping plot for: Sreedhanya Catering Service
new name is Sreedhanya Catering Service
Scraping plot for: Palthu Janwar
new name is Palthu Janwar
Scraping plot for: Sundari Gardens
new name is Sundari Gardens
Scraping plot for: Ottu
new name is Ottu (film)
Scraping plot for: Pathonpatham Noottandu
new name is Pathonpatham Noottandu
Scraping plot for: Kotthu
new name is Kotthu
Scraping plot for: Ormakalil
new name is Ormakalil
Scraping plot for: Mei Hoom Moosa
new name is Mei Hoom Moosa
Scraping plot for: Rorschach
new name is Rorschach (film)
Scraping plot for: Varaal
new name is Varaal (film)
Scraping plot for: Vichithram
new name is Vichithram
Scraping plot for: Padavettu
new name is Padavettu
Scraping plot for: Appan
new name is Appan (film)
Scraping plot for: Autorickshawkarante Bharya
new name is Autorickshawkarante Bharya
Scraping plot for: Jaya Jaya Jaya Jaya Hey
new